In [198]:
import re
import pandas as pd
import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NamesExtractor,
    NewsNERTagger,
    PER,
    Doc
)
import eli5
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_box_markup as show_markup
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from itertools import chain
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_ru = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /home/medic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
test_data = pd.read_excel('./test_hotels.xlsx')
train_data = pd.read_excel('./train_hotels.xlsx')

In [74]:
# for create classes in our dataset 
def create_sentiment(x):
    sentiment = 0
    if x > 3:
        sentiment = 1
    else:
        sentiment = 0
    
    return sentiment

In [66]:
train_data['sentiment'] = train_data.apply(lambda x: create_sentiment(x['sentiment']), axis = 1)
train_data['sentiment'].unique()

array([1, 0])

In [172]:
test_data['sentiment'] = test_data.apply(lambda x: create_sentiment(x['sentiment']), axis = 1)
train_data['sentiment'].unique()

array([1, 0])

In [173]:
# clear Nan data in datasets
train_data.dropna(subset = ['text'], inplace = True)
test_data.dropna(subset = ['text'], inplace = True)

In [175]:
train_data.info(), test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50175 entries, 0 to 50327
Data columns (total 3 columns):
sentiment    50175 non-null int64
text         50175 non-null object
predict      50175 non-null int64
dtypes: int64(2), object(1)
memory usage: 1.5+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6876 entries, 0 to 6875
Data columns (total 2 columns):
sentiment    6876 non-null int64
text         6876 non-null object
dtypes: int64(1), object(1)
memory usage: 161.2+ KB


(None, None)

#### Начнем классификацию текстов. 
Первым попробуем наивный подход - проверка в тексте слов характеных для положительных отзывов, и соотвественно отрицательных. 
Составим массив положительных и отрицательных слов для нашей "наивной" классификации.
Воспользуемся CountVectorizer замечательной библиотеки sklearn

In [106]:
def clean_text(text):
    new_text = text.lower()
    new_text = [word for word in new_text.split() if len(word) > 1]
    new_text = [word for word in new_text if word not in stopwords_ru]
    new_text = [re.sub(r'[^\w\s]', '', word) for word in new_text]
    new_text = [re.sub(r'[^а-яА-Я0-9]', '', word) for word in new_text]
    new_text = [re.sub(r'[^а-яА-Я]', '', word) for word in new_text]
    new_text = []
    return new_text

In [116]:
negative_list = [word for word in list(train_data['text'][train_data['sentiment'] == 0])]
negative_list = clean_text(negative_list[0])

In [120]:
# most freq negative words
count_vectorizer_neg = CountVectorizer()
counts_neg = count_vectorizer_neg.fit_transform(negative_list)
negative_words = count_vectorizer_neg.get_feature_names()

In [121]:
# most freq positive words
positive_list = [word for word in list(train_data['text'][train_data['sentiment'] == 1])]
positive_list = clean_text(positive_list[0])

count_vectorizer_pos = CountVectorizer()
counts_pos = count_vectorizer_pos.fit_transform(positive_list)
positive_words = count_vectorizer_pos.get_feature_names()

In [162]:
diff = list(set(positive_list) - set(negative_list))
positive_list = list(set(positive_list) - set(diff))
negative_list = list(set(negative_list) - set(diff))

In [164]:
positive_words[:30], negative_words[:30]

(['автобусе',
  'бассейном',
  'брали',
  'быстро',
  'добраться',
  'достаточно',
  'достойный',
  'завтраки',
  'займет',
  'интернетом',
  'крыше',
  'метро',
  'минут',
  'могу',
  'номерами',
  'отель',
  'очень',
  'пешком',
  'поэтому',
  'прекрасной',
  'прекрасными',
  'прогулка',
  'сказать',
  'террасой',
  'уборкой',
  'хорошим',
  'центра'],
 ['автомобильную',
  'ад',
  'администратора',
  'бассейн',
  'бассейноми',
  'берет',
  'берут',
  'бокала',
  'большим',
  'видами',
  'вина',
  'воздух',
  'вход',
  'вызвать',
  'выходят',
  'далеко',
  'двумя',
  'деньги',
  'дом',
  'дорогу',
  'евро',
  'закрытым',
  'зал',
  'звонок',
  'зона',
  'коекак',
  'которые',
  'минутпросьба',
  'мыли',
  'мыльных'])

In [125]:
def choice_negative_or_positiv(text):
    positives = sum(word in text for word in positive_words)
    negatives = sum(word in text for word in negative_words)
    
    if positives > negatives:
        return 1
    else:
        return 0

In [165]:
sample = train_data['text'][3:4]
choice_negative_or_positiv(sample)

0

In [167]:
train_data['predict'] = train_data.apply(lambda x: choice_negative_or_positiv(x['text']), axis = 1)

In [168]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(list(train_data['sentiment']), list(train_data['predict']))
accuracy

0.18282012954658694

Мы получили совсем плохой результат. Давайте попробуем его улучшить уже с помощью методик машинного обучения.

### LogisticRegression 
попробуем самую простую модель logistirRegression из библиотеки sklearn 

In [176]:
vectorizer = CountVectorizer()
classifier = LogisticRegression()

model_lr = Pipeline([('vectorizer', vectorizer), ('classifier', classifier)])

model_lr.fit(train_data['text'], train_data['sentiment'])

/home/medic/python36/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', LogisticRegression())])

In [178]:
lr_pred = model_lr.predict(test_data['text'])
lr_res = accuracy_score(test_data['sentiment'], lr_pred)
lr_res

0.8913612565445026

Как всегда, простая модель логистической регрессии показывает очень хороший результат. С таким результататом уже можно работать и показывать хорошие результаты.

Посмотрим на наши что же попадает у нас в положительные и что в отрицательные отзывы:

In [183]:
eli5.show_weights(classifier,  vec = vectorizer, top = 40)

Weight?,Feature
+1.808,современная
+1.669,приличная
+1.600,непосредственной
+1.576,рекомендуем
+1.469,довольны
+1.460,<BIAS>
+1.453,благодарность
+1.430,устали
+1.418,впечатляет
+1.417,устраивает


Отличное разделение!

Теперь можно попробовать сделать тоже самое но уже с учетом частоты встречаемости слов. Так слова в тексте встречаются с разной частотой - какие-то чаще, какие-то реже, следовательно у них будут разные веса, и это тоже можно учитывать в нашей модели. Для того чтобы учитывать частоту встречаемости слов можно использовать TfidVecrorizer из все той же библиотеки sklearn. Заменим CountVecrorizer на TfidVecrorizer и проведем новое обучение и проверим результат. 

In [299]:
vectorizer2 = TfidfVectorizer()
classifier2 = LogisticRegression()

model_lr2 = Pipeline([('vectorizer', vectorizer2), ('classifier', classifier2)])

model_lr2.fit(train_data['text'], train_data['sentiment'])

lr_pred2 = model_lr2.predict(test_data['text'])
lr_res2 = accuracy_score(test_data['sentiment'], lr_pred2)
lr_res2


/home/medic/python36/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9006689936009308

Теперь можно подключить контекст. В качестве иструмента для работы с контекстом можно использовать n-gram слов,
чтобы модель смотрела не отдельное слово а на их сочание. В некоторых случаях это сильно может сказаться
на конечно результате.

Попробуем добавить ngram из двух слов в нашу новую модель.

In [300]:
vectorizer3 = TfidfVectorizer(ngram_range=(1,2))
classifier3 = LogisticRegression()

model_lr3 = Pipeline([('vectorizer', vectorizer3), ('classifier', classifier3)])

model_lr3.fit(train_data['text'], train_data['sentiment'])

lr_pred3 = model_lr3.predict(test_data['text'])
lr_res3 = accuracy_score(test_data['sentiment'], lr_pred3)
lr_res3

0.8979057591623036

Результат стал немного лучше, подход с изменением количества слов в векторизации имеет место быть.

Еще один поход - брать не g-граммы слов, а n-граммы симолово от слов. Попробуем этот поход на той же схеме vectorizer и classifier добавив параметры analyzer = 'char'.

In [192]:
vectorizer4 = TfidfVectorizer(analyzer = 'char', ngram_range=(1,6), max_features = 40000)
classifier4 = LogisticRegression()

model_lr4 = Pipeline([('vectorizer', vectorizer4), ('classifier', classifier4)])

model_lr4.fit(train_data['text'], train_data['sentiment'])

lr_pred4 = model_lr4.predict(test_data['text'])
lr_res4 = accuracy_score(test_data['sentiment'], lr_pred4)
lr_res4

0.8999418266433973

#### Лематтизация 
Лемматизация позволяет нам приводить слова к нормальной форме. Проверим, влияет ли лемматизация на результат обучения и предсказания модели.

Добавим в наш дата фрейм колонку с лемматизированным текстом, и поробуем обучение и предсказание на этих данных.

In [200]:
# use lib natasha for create lemmatizer and lemmatize word in our dataset
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
name_extractor = NamesExtractor(morph_vocab)

In [247]:
def lemmatization(text):
    output = []
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    for _ in doc.tokens:
        if (len(_.lemma) > 2) and (_.lemma not in stopwords_ru):
            output.append(_.lemma)
    
    return ' '.join(output)

In [248]:
train_data['lemmatize_text'] = train_data.apply(lambda x: lemmatization(x['text']), axis = 1)
test_data['lemmatize_text'] = test_data.apply(lambda x: lemmatization(x['text']), axis = 1)

In [249]:
train_data

,sentiment,text,predict,lemmatize_text
0,1,"Очень достойный отель с прекрасными номерами, ...",1,очень достойный отель прекрасный номер хороший...
1,1,"Остановились в Барселоне проездом, т.к. нужно ...",0,остановиться барселона проезд нужный посетить ...
2,1,Типичная сетевая гостиница. Главный плюс-шикар...,0,типичный сетевой гостиница главный плюс-шикарн...
3,0,"Начнем с того, что в этом отеле не берут деньг...",0,начать отель брать деньга воздух звонок телефо...
4,1,"Отель находится в отдалении от центра,но пешко...",0,отель находиться отдаление центр пешком дойти ...
...,...,...,...,...
50323,1,Была в этом отеле всего одни сутки в конце пое...,0,отель сутки конец поездка отель понравиться бе...
50324,0,Местоположение отличное. Сервис хороший. Завтр...,0,местоположение отличный сервис хороший завтрак...
50325,1,Отдыхали вдвоем с мужем в январе 2015. Отель п...,0,отдыхать вдвоем муж январь 2015 отель понравит...
50326,1,Отдыхали с друзьями впятером. Больше всего пон...,0,отдыхать друг впятером весь понравиться террит...


In [250]:
# save dataset
train_data.to_pickle('train_data_hotels.pickle')
test_data.to_pickle('test_data_hotels.pickle')

In [301]:
# regresion in lemmatisation
vectorizer_lem = TfidfVectorizer()
classifier_lem = LogisticRegression()

model_lem = Pipeline([('vectorizer', vectorizer_lem), ('classifier', classifier_lem)])

model_lem.fit(train_data['lemmatize_text'], train_data['sentiment'])

lr_pred_lem = model_lem.predict(test_data['lemmatize_text'])
lr_res_lem = accuracy_score(test_data['sentiment'], lr_pred_lem)
lr_res_lem

/home/medic/python36/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9029959278650378

лемматизация помогла еще повысить наш  результат на несколько процентных пунктов.

Попробуем еще одну технику из NLP приема - заменим распознанные сущности NER на их теги и посмотрим на результат.

In [292]:
def ner_search(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        text = text.replace(span.text, span.type)
    return text

In [295]:
train_data['text_NER'] = train_data.apply(lambda x: ner_search(x['text']), axis = 1)
test_data['text_NER'] = test_data.apply(lambda x: ner_search(x['text']), axis = 1)

In [302]:
# regresion in NER tags
vectorizer_ner = TfidfVectorizer()
classifier_ner = LogisticRegression()

model_ner = Pipeline([('vectorizer', vectorizer_ner), ('classifier', classifier_ner)])

model_ner.fit(train_data['text_NER'], train_data['sentiment'])

lr_pred_ner = model_ner.predict(test_data['text_NER'])
lr_res_ner = accuracy_score(test_data['sentiment'], lr_pred_ner)
lr_res_ner

/home/medic/python36/lib64/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9013961605584643

<table>
    <head>
        <tr >
            <td>№</td>
            <td>Модель</td>   
            <td>Результат (accuracy)</td>
        </tr>
    </head>
    <tr>
        <td>1</td>
        <td>"Наивная модель" - просто взяли слова из текста и посчитали их количество</td>
        <td>0.18</td>
    </tr>
    <tr>
        <td>2</td>
        <td>Простая LogicRegression - обычный CountVectorizer() без дополнительных параметров </td>
        <td>0.891361</td>
    </tr>
    <tr>
        <td>3</td>
        <td>Простая LogicRegression - TfidfVectorizer() учитывающий частоту встречающихся слов</td>
        <td>0.900668</td>
    </tr>
    <tr>
        <td>4</td>
        <td>Простая LogicRegression - TfidfVectorizer() c n-gramm в два слова</td>
        <td>0.900668</td>
    </tr>
    <tr>
        <td>5</td>
        <td>Простая LogicRegression - TfidfVectorizer() с n-gramm из символов</td>
        <td>0.899941</td>
    </tr>
    <tr>
        <td>6</td>
        <td>Простая LogicRegression - TfidfVectorizer() c lemmatizer из библиотеке natasha</td>
        <td>0.902995</td>
    </tr>
     <tr>
        <td>7</td>
        <td>Простая LogicRegression - TfidfVectorizer() c NER типом токена вместо вместо самих токенов</td>
        <td>0.901396</td>
    </tr>
    </table>

#### Выводы:
1. Наивный подход ожидаемо работает хуже всех. Однако тут есть поле для улучшения результата. И при достаточном количестве времени результ можно поднять до 0.6 а может быть даже и выше.
2. Не плохой результат показал самый просто ML метод с простым CountVectorizer. Впрочем, как обычно, простые классические методы ML за частую показывают результаты не хуче чем новые методы основанные на нейронка и тербующие большие мощности для своей работы.
3. Лучший результат показала LR с лемматизацией. Результат закономерен так как тут мы получаем наиболее чистые данные для модели.
4. На втором месте по точности определения модель LR с данными в которых заменили определенные перед этим NER сущности и заменили их на обочнающие их типы.
5. Для русского языка лучше рабоет модель c n-gramm для нескольких слов, чем n-gramm для символов. Вот такая особенность у русского языка, в английском языке бывает наоборот. Ну там и стеммизация работает не плохо, которая в русском языке не работает совсем.